# Source Code

## Import library

In [1]:
import os
import time
from operator import itemgetter

In [2]:
W: int #capable weight
m: int #number of classes
w = [] #weight of each item
v = [] #value of each item
c = [] #class of each item

## Read input files

In [3]:
# set path for input files
input_dir='data/input/' 
for file in os.listdir(input_dir):
    # open each file
    with open(input_dir+file) as f:
        # check if that file is .txt file or not
        if (file[-4:]!='.txt'): continue
        # read 5 input strings from file to variables
        capacity,class_num,weight,val,label=f.readlines()
        # set value for W and m
        W,m=int(capacity),int(class_num)
        # set value for w 
        w=weight.replace(' ','').replace('\n','')
        w=[float(num) for num in weight.split(',')]
        # set value for v
        v=val.replace(' ','').replace('\n','')
        v=[float(num) for num in val.split(',')]
        # set value for c
        c=label.replace(' ','').replace('\n','')
        c=[float(num) for num in label.split(',')]

## Implementation

In [4]:
# initialize a class to store information of decision tree
class Node:
    def __init__(self, index, value, weight):
        # path: path from the root to the node
        # index: index of this node in the given item list
        # value: value of nodes on path from root to this node (including this node)
        # weight: weight of nodes on path from root to this node (including this node)
        self.path = []
        self.index = index
        self.value = value
        self.weight = weight

In [5]:
# check if the considered solution satisfies the condition:
# select at least one item from each class or not
def check_knapsack (knapsack, m, c):
    knapsack_class = set()
    for i in knapsack:
        knapsack_class.add(c[i])
    # if the set of items contains all given class, then we return true
    if (len(knapsack_class) == m): return True
    #else return false
    return False

In [6]:
# function to sort item in term of the "profit" (value / weight) of each item in descending order
def sort_values_and_weights(values, weights, n, c):
    sortList = []
    for i in range(n):
        sortList.append((values[i], weights[i], values[i]/weights[i], i, c[i]))

    # sort by the "profit" value of item, but the output list is sorted in ascending order
    sortList = sorted(sortList, key=itemgetter(2))
    # so we reverse it to have a sorted list in descending order
    sortList.reverse()

    # initialize list to get essential value from the sorted list
    sorted_values = []
    sorted_weights = []
    indexes = []
    classes = []

    for a in sortList:
        # get items' values in sorted list
        sorted_values.append(a[0])
        # get items' weights in sorted list
        sorted_weights.append(a[1])
        # get items' indexes in sorted list
        indexes.append(a[3])
        # get items' classes in sorted list
        classes.append(a[4])

    return sorted_values, sorted_weights, indexes, classes

In [7]:
# Returns bound of value in subtree rooted with 'node'
def bound(node, n, W, values, weights):

    # if weight overcomes the knapsack capacity, return
    # 0 as expected bound
    if node.weight >= W:
        return 0

    else:
        # initialize bound on value by current value
        bound = node.value
        # initialize essential information for calculating
        total_weight = node.weight
        j = node.index + 1

        # start from the item after current item and sum up all value until
        # reaching the last item or total weight exceed the capability
        # these codes are to calculate bound and find the first item that
        # does not fit completely in the knapsack
        while (j < n) and total_weight + weights[j] <= W:
            total_weight += weights[j]
            bound += values[j]
            j += 1

        # if it's still not the last item
        if j < n:
            # then calculate the fraction of item's value that 
            # still fits in the knapsack 
            bound += (W - total_weight) * (values[j]/weights[j])

        return bound

In [8]:
# implement the algorithm
def branch_and_bound_knapsack(W, values, weights, c, m):

    n = len(values)

    # sorting item on basis of value per weight
    values, weights, indexes, c = sort_values_and_weights(values, weights, n, c)

    # initializing a queue for traversing
    pq = []
    # initializing a list to store result
    result = []

    # start with a node of index -1 to
    # act as previous node
    v = Node(-1, 0, 0)
    # current node
    node = Node(0, 0, 0)
    # initializing highest value as 0
    highest_value = 0

    pq.append(v)

    while pq:
        
        # get the first node in the queue
        v = pq.pop(0)

        # if tree does not have any node then 
        # current node start with item index 0
        if(v.index == -1):
            node.index = 0

        # if it's the last item, then stop branching
        if node.index == n-1:
            continue

        # if it's not the last node, then increase the index
        node.index = v.index + 1

        # compute the value and weight of current node
        # by adding current node's value and weight to 
        # the accumulated value and weight stored in previous node
        node.value = v.value + values[node.index]
        node.weight = v.weight + weights[node.index]

        # set the path from root to current node
        node.path = v.path.copy()
        node.path.append(indexes[node.index])

        # if total weight until now is less than the capability
        # and total value is higher than current highest value
        # then update highest value and result path
        if node.weight <= W and node.value > highest_value:
            if check_knapsack(node.path, m, c):
                highest_value = node.value
                result = node.path

        # get the upper bound to decide whether we should continue
        # add more nodes and traverse on this branch that have current node
        node.bound = bound(node, n, W, values, weights)

        # if the upper bound is not higher than current highest value
        # then it is useless to keep considering this branch
        # else we push this node into queue for further consideration
        if node.bound > highest_value:
            # this node represent child node that assume we will
            # take the current item
            pq.append(node)

        # create a child node indicate branch that does not contain 
        # current node
        node = Node(node.index, v.value, v.weight)
        # do the same thing with this node (calculate upper bound
        # and check whether we should add this node into queue)
        node.bound = bound(node, n, W, values, weights)
        node.path = v.path.copy()

        if node.bound > highest_value:
            pq.append(node)

    # return result
    return highest_value, result

In [10]:
start = time.time()
(result, highest_value) = (branch_and_bound_knapsack(W, v, w, c, m))
end = time.time()

# print the result list, high value and total time needed to run the algorithm
print(result, highest_value, (end - start))

117.0 [1, 4, 9] 0.0009949207305908203


# Report

## Algorithm Description

This idea of this algorithm is based on recursively branching and pruning the search tree by calculating an upper bound, which is the maximum potential value, to consider whether or not we should continue traverse on that branch. If it's not worth further consideration, we can prune that branch to save time.

The idea after calculating upper bound, which is the maximum potential value that branch can get, is that: if the upper bound for some node is not exceed the highest value, meaning that it not worth keeping this branch since we will never get the optimal solution on this branch, thus we can safely discarded it from the tree. 

## Pseudo Code

*This is pseudo code for function checking if the list contains all classes or not*

```
check_knapsack(knapsack: list, m, c) return bool
    knapsack_class is set
    FOR i IN knapsack
        BEGIN
            knapsack_class add c[i]
        END
    IF (size knapsack_lass = m)
        return True
        ENDIF
    return False
```

*This is pseudo code for function sorting items based on value per weight*

```
sort_value_and_weight(values, weights, n, c) return sorted_values, sorted_weights, indexes, classes
    sortList <- []
    FOR i < n:
        BEGIN
        sortList add (values[i], weights[i], values[i]/weights[i], i, c[i])
        END

    sort sortList based on the third element in list
    reverse sortList

    sorted_values <- []
    sorted_weights <- []
    indexes <- []
    classes <- []

    FOR a IN sortList:
        BEGIN
        sorted_values <- add a[0]
        sorted_weights <- add a[1]
        indexes <- add a[3]
        classes <- add a[3]
        END

    return sorted_values, sorted_weights, indexes, classes
```
*This is the pseudo code for function calculating bound*

```
bound(node, n, W, values, weights) return int
    IF node.weight >= W
        return 0
        ENDIF

    else:
        bound <- node.value
        total_weight <- node.weight
        j <- node.index + 1

        WHILE (j < n) AND total_weight + weights[j] <= W
            total_weight += weights[j]
            bound += values[j]
            j += 1

        if j < n:
            bound += (W - total_weight) * (values[j]/weights[j])
            ENDIF

        return bound
```

*This is the pseudo code for function implementing branch and bound algorithm*

```
branch_and_bound_knapsack(W, values, weights, c, m) return highest_value, result

    n <- size of values

    values, weights, indexes, c <- sort_values_and_weights(values, weights, n, c)

    pq <- []
    result = []

    v <- Node(-1, 0, 0)
    node <- Node(0, 0, 0)
    highest_value <- 0

    pq add v

    WHILE pq:
        
        v pop first element in pq

        if(v.index = -1)
            node.index <- 0
            ENDIF

        if node.index = n-1
            CONTINUE
            ENDIF

        node.index <- v.index + 1

        node.value <- v.value + values[node.index]
        node.weight <- v.weight + weights[node.index]

        node.path copy v.path
        node.path add indexes[node.index]

        IF node.weight <= W AND node.value > highest_value
            IF check_knapsack(node.path, m, c)
                highest_value <- node.value
                result <- node.path
                ENDIF
            ENDIF

        node.bound <- bound(node, n, W, values, weights)

        if node.bound > highest_value
            pq add node
            ENDIF

        node <- Node(node.index, v.value, v.weight)
        node.bound <- bound(node, n, W, values, weights)
        node.path copy v.path

        if node.bound > highest_value
            pq add node
            ENDIF

    return highest_value, result
```

## Algorithm Explanation

To implement this algorithm, first we have to sort the item list on basis of value / weight to make sure that we prioritize those that have larger value and smaller weight so that we can add them first when calculating bound and not miss them. 

Then we implement a function to calculate bound. The idea is that we will continuously add item's value in the sorted list until it reach the capability of the knapsack. If the knapsack still has some weights left, we calculate the fraction of the next item that still fits in the knapsack. In the end, we sum up all values from the current item to the first n-1 items and a fraction of item s.

$$\text{bound(node s) }= v[s] + \sum_{i=s+1}^{n}v[i]+((W - \sum_{i=s+1}^{n}w[i]) / w[n+1]) *  v[n+1]$$

where the nodes from index s+1 to n denote items can be fully fit into the
knapsack and the node at n+1 is fit into the knapsack whose capacity is W with a fractional value. 

For the main function implement the algorithm, we first start with an empty queue and dummy node, then perform breadth-first traversal on the remaining nodes from the root node. Each node will branch out two child nodes represent for decisions of whether we take the parent node or not. At each node update the current weights and values, calculate the bound and update the global optimum or prune if necessary